In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta-multitask')  # or 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta-multitask')  # or 'BM-K/KoSimCSE-bert-multitask'

sentences = ['치타가 들판을 가로 질러 먹이를 쫓는다.',
             '치타 한 마리가 먹이 뒤에서 달리고 있다.',
             '원숭이 한 마리가 드럼을 연주한다.']

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])  # 84.09
# '치타가 들판을 가로 질러 먹이를 쫓는다.' @ '치타 한 마리가 먹이 뒤에서 달리고 있다.'
score02 = cal_score(embeddings[0][0], embeddings[2][0])  # 23.21
# '치타가 들판을 가로 질러 먹이를 쫓는다.' @ '원숭이 한 마리가 드럼을 연주한다.'

print(score01, score02)

tensor([[84.0977]], grad_fn=<MulBackward0>) tensor([[23.2168]], grad_fn=<MulBackward0>)


KoSBERT를 활용해서 문장 간의 유사도를 구할 수 있음의 예시 -> score가 높을 수록 유사도가 높다.

가지고 있는 데이터의 헤드라인과 본문을 일부 대입시켜 테스트해봤을 때,  
관련있는 기사의 경우에는 82.77, 57.30이 나온 반면에  
관련없는 기사의 경우에는 32.64, 28.28이 나와 변별이 가능할 것으로 보인다.

In [ ]:
sentences = ['\\\"목구멍이 포도청\\\"…빚투 논란 이혁재 유튜브로 복귀',
             '방송인 이혁재가 \'빚투\' 논란 후 1년 만에 개인 유튜브 채널을 통해 복귀했다',
             '이혁재는 1999년 MBC 공채 개그맨으로 데뷔해 각종 예능 프로그램에 출연하며 인기를 얻었다.']

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])
# '\\\"목구멍이 포도청\\\"…빚투 논란 이혁재 유튜브로 복귀' @ '방송인 이혁재가 \'빚투\' 논란 후 1년 만에 개인 유튜브 채널을 통해 복귀했다'
score02 = cal_score(embeddings[0][0], embeddings[2][0])
# '\\\"목구멍이 포도청\\\"…빚투 논란 이혁재 유튜브로 복귀' @ '이혁재는 1999년 MBC 공채 개그맨으로 데뷔해 각종 예능 프로그램에 출연하며 인기를 얻었다.'

print(score01, score02)

tensor([[82.7740]], grad_fn=<MulBackward0>) tensor([[57.3056]], grad_fn=<MulBackward0>)


In [ ]:
sentences = ['쯔위, 박진영 사과… 트와이스 중국활동 정지',
             '‘안테나 식구’ 유희열과 유재석이 신종 코로나바이러스 감염증(코로나19) 완치 판정을 받았다.',
             '앞서 안테나 대표 유희열은 코로나19 백신 2차 접종까지 완료했지만 지난 10일 양성 판정을 받았다.']

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])
# '쯔위, 박진영 사과… 트와이스 중국활동 정지' @ '‘안테나 식구’ 유희열과 유재석이 신종 코로나바이러스 감염증(코로나19) 완치 판정을 받았다.'
score02 = cal_score(embeddings[0][0], embeddings[2][0])
# '쯔위, 박진영 사과… 트와이스 중국활동 정지' @ '앞서 안테나 대표 유희열은 코로나19 백신 2차 접종까지 완료했지만 지난 10일 양성 판정을 받았다.'

print(score01, score02)

tensor([[32.6460]], grad_fn=<MulBackward0>) tensor([[28.2838]], grad_fn=<MulBackward0>)
